<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/binomial_fdm_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

class BinomialTreeBermudan:
    def __init__(self, S0, K, T, r, sigma, n_steps, exercise_dates):
        """
        S0: Initial stock price
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free interest rate
        sigma: Volatility
        n_steps: Number of time steps in the binomial tree
        exercise_dates: List of exercise dates (as fractions of T)
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_steps = n_steps
        self.exercise_dates = exercise_dates
        self.dt = torch.tensor(T/n_steps, dtype=torch.float32)
        self.u = torch.exp(sigma * torch.sqrt(self.dt))
        self.d = 1 / self.u
        self.p = (torch.exp(r * self.dt) - self.d) / (self.u - self.d)

    def build_tree(self):
        # Initialize asset prices at maturity
        self.S = torch.zeros(self.n_steps + 1, self.n_steps + 1)
        self.S[0, 0] = self.S0

        for i in range(1, self.n_steps + 1):
            self.S[0, i] = self.S[0, i - 1] * self.u
            for j in range(1, i + 1):
                self.S[j, i] = self.S[j - 1, i - 1] * self.d

    def option_price(self, option_type='call'):
        # Initialize option values at maturity
        self.V = torch.zeros_like(self.S)
        if option_type == 'call':
            self.V[:, -1] = torch.maximum(self.S[:, -1] - self.K, torch.tensor(0.0))
        elif option_type == 'put':
            self.V[:, -1] = torch.maximum(self.K - self.S[:, -1], torch.tensor(0.0))
        else:
            raise ValueError("option_type must be 'call' or 'put'")

        # Backward induction
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                self.V[j, i] = torch.exp(-self.r * self.dt) * (self.p * self.V[j, i + 1] + (1 - self.p) * self.V[j + 1, i + 1])

                # Check if current time step is an exercise date
                if i * self.dt in self.exercise_dates:
                    if option_type == 'call':
                        exercise_value = torch.maximum(self.S[j, i] - self.K, torch.tensor(0.0))
                    else:
                        exercise_value = torch.maximum(self.K - self.S[j, i], torch.tensor(0.0))
                    self.V[j, i] = torch.maximum(self.V[j, i], exercise_value)

        return self.V[0, 0]

    def calculate_greeks(self, option_type='call'):
        # Calculate Delta
        delta = (self.V[0, 1] - self.V[1, 1]) / (self.S[0, 1] - self.S[1, 1])

        # Calculate Gamma
        gamma = ((self.V[0, 2] - self.V[1, 2]) / (self.S[0, 2] - self.S[1, 2]) -
                 (self.V[1, 2] - self.V[2, 2]) / (self.S[1, 2] - self.S[2, 2])) / (0.5 * (self.S[0, 2] - self.S[2, 2]))

        # Calculate Theta
        theta = (self.V[0, 1] - self.V[0, 0]) / self.dt

        # Calculate Vega (using a small change in volatility)
        sigma_shift = 0.01
        tree_shifted = BinomialTreeBermudan(self.S0, self.K, self.T, self.r, self.sigma + sigma_shift, self.n_steps, self.exercise_dates)
        tree_shifted.build_tree()
        price_shifted = tree_shifted.option_price(option_type)
        vega = (price_shifted - self.option_price(option_type)) / sigma_shift

        # Calculate Rho (using a small change in interest rate)
        r_shift = 0.01
        tree_shifted = BinomialTreeBermudan(self.S0, self.K, self.T, self.r + r_shift, self.sigma, self.n_steps, self.exercise_dates)
        tree_shifted.build_tree()
        price_shifted = tree_shifted.option_price(option_type)
        rho = (price_shifted - self.option_price(option_type)) / r_shift

        return delta, gamma, theta, vega, rho

# Example usage
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
sigma = 0.2
n_steps = 100
exercise_dates = [0.25, 0.5, 0.75]

tree = BinomialTreeBermudan(S0, K, T, r, sigma, n_steps, exercise_dates)
tree.build_tree()
option_price = tree.option_price(option_type='call')
delta, gamma, theta, vega, rho = tree.calculate_greeks(option_type='call')

print(f"Option Price: {option_price:.4f}")
print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Theta: {theta:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Rho: {rho:.4f}")